In [3]:
from drowsiness_detection.data import load_raw_60_sec_data
import numpy as np
from drowsiness_detection import config
from joblib import dump
import time
from drowsiness_detection.models import build_cnn_model

In [2]:
data = load_raw_60_sec_data()

data.X_train = np.concatenate([data.X_train, data.X_val])
data.y_train = np.concatenate([data.y_train, data.y_val])


In [20]:
model_name = "cnn"

best_params = {
    "input_shape": (20, 1800, 7),
    "dropout_rate": 0.725,
    "kernel_size": 5,
    "learning_rate": 0.009,
    "num_conv_layers": 2,
    "num_filters": 44,
    "pooling": "max",
    "stride": 3
  }
model = build_cnn_model(**best_params)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1800, 7)]         0         
                                                                 
 conv1d_2 (Conv1D)           (None, 600, 44)           1584      
                                                                 
 batch_normalization_2 (Batc  (None, 600, 44)          176       
 hNormalization)                                                 
                                                                 
 re_lu_2 (ReLU)              (None, 600, 44)           0         
                                                                 
 conv1d_3 (Conv1D)           (None, 200, 44)           9724      
                                                                 
 batch_normalization_3 (Batc  (None, 200, 44)          176       
 hNormalization)                                           

In [18]:
history = model.fit(x=data.X_train, y=data.y_train, validation_data=(data.X_test, data.y_test) , verbose=1, class_weight={0: 0.84, 1: 1.14},
                    epochs=150, batch_size=20)


Epoch 1/150
1283/1283 [==============================] - 17s 13ms/step - loss: 0.6551 - accuracy: 0.5308 - val_loss: 0.6677 - val_accuracy: 0.5991
Epoch 2/150
1283/1283 [==============================] - 18s 14ms/step - loss: 0.6518 - accuracy: 0.5222 - val_loss: 0.6929 - val_accuracy: 0.4613
Epoch 3/150
1283/1283 [==============================] - 17s 13ms/step - loss: 0.6516 - accuracy: 0.5263 - val_loss: 0.6515 - val_accuracy: 0.5991
Epoch 4/150
1283/1283 [==============================] - 17s 13ms/step - loss: 0.6476 - accuracy: 0.5068 - val_loss: 0.7017 - val_accuracy: 0.5410
Epoch 5/150
1283/1283 [==============================] - 17s 13ms/step - loss: 0.6431 - accuracy: 0.5222 - val_loss: 0.7098 - val_accuracy: 0.4026
Epoch 6/150
1283/1283 [==============================] - 17s 13ms/step - loss: 0.6377 - accuracy: 0.5121 - val_loss: 0.6908 - val_accuracy: 0.6063
Epoch 7/150
1283/1283 [==============================] - 18s 14ms/step - loss: 0.6338 - accuracy: 0.5263 - val_loss: 0

In [21]:
time_string = time.strftime('%Y%m%d-%H%M%S')
model.save(str(config.MODEL_DIR_PATH.joinpath(f"{model_name}_{time_string}.h5")))


In [17]:
dump(history, str(config.MODEL_DIR_PATH.joinpath(f"histories/{model_name}_{time_string}.joblib")))

INFO:tensorflow:Assets written to: ram://8c12d3b6-f35a-4fd1-8f21-eb8a5ff2afaa/assets


['/home/tim/IM/data/models/60sec/histories/cnn_20220728-151804.joblib']

In [10]:
y_pred_test = model.predict(data.X_test)
y_pred_train = model.predict(data.X_train)

In [11]:
print(np.mean((y_pred_test > .5) == data.y_test))


0.44231875344246974


In [12]:
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_test_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_test)
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_train_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_train)
